In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score

In [22]:
# Upload repairs data
dtp = pd.read_excel('C:/Users/sgulbin/Work/_Выгрузки_и_Расчеты/accidents_4_10_2020.xlsx')
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/18/2020 12:00:00 AM', '3/18/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/20/2020 12:00:00 AM', '3/20/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='5/24/2020 12:00:00 AM', '3/24/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='7/23/2020 12:00:00 AM', '3/23/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'], dayfirst=False, errors = 'coerce')
dtp.sort_values('Дата и время ДТП', inplace=True)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'])
dtp = dtp[dtp['Телефон водителя'].notnull()]
dtp['Телефон водителя'] = dtp['Телефон водителя'].apply(int)
dtp = dtp[dtp['ТС'].notnull()]
dtp.reset_index(drop=True, inplace = True)
dtp['ТС'] = dtp['ТС'].str.replace(' ', '')
dtp = dtp[['Дата и время ДТП', 'Регион','Телефон водителя', 'Виновность', 'ТС', 'Модель автомобиля', 'POOL', 'Тариф','Ремонт не производился','Тип ДТП', 'Вид страхования', 'Пострадавшие','Сумма по заказ-наряду','Выставлено', 'Страховая компания сумма выплаченная']].rename(columns={
    'Дата и время ДТП': 'dt',
    'Регион': 'region',
    'Телефон водителя': 'phone',
    'Виновность': 'Responsibility',
    'ТС': 'license_plate',
    'Модель автомобиля': 'vehicle_model',
    'POOL': 'pool',
    'Тариф': 'tariff',
    'Ремонт не производился': 'no repair conducted',
    'тип ДТП': 'accident type',
    'Вид страхования': 'insurance type',
    'Пострадавшие': 'injured',
    'Сумма по заказ-наряду': 'target_amount',
    'Выставлено': 'Vystavleno',
    'Страховая компания сумма выплаченная': 'insurance paid sum'
})

In [23]:
# Total end of March validation
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation_v2/aggregated_probs_till_march2020.csv')
# users = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v7.csv', low_memory = False)
users = pd.read_csv('C:/Users/sgulbin/Work/Data/users_3_18-3_29.csv')
df = pd.merge(df, users, left_on = 'UserId', right_on = 'user_ext', how = 'left')
df = df.loc[pd.isnull(df.login) == False]
# users = users[['login', 'user_ext']]

# Changing "phone" format from int64 to string (object) for successful dataframes merge
# dtp.phone = dtp.phone.astype(str)
dtp = pd.merge(dtp, users, left_on = 'phone', right_on = 'login', how = 'left')

# Only guilty and unknown
dtp = dtp.loc[(dtp.Responsibility != 'Фейк')&(dtp.Responsibility != 'Угон')&(dtp.Responsibility != 'Не виновен')]

# Data formatting
dtp['user_ext'] = dtp.user_ext.fillna(0)
df.UserId = df.UserId.astype(int)

# Splitting dtp data into periods for validation
# March
dtp_march = dtp.loc[(dtp.dt >= '2020-03-18') & (dtp.dt <= '2020-03-29')]
dtp_march = dtp_march[['dt','user_ext']]
dtp_march['march'] = 1

# Marking users with accidents
df = pd.merge(df,dtp_march, left_on = 'UserId', right_on = 'user_ext', how = 'left')
df['march'] = df['march'].fillna(0)

# Running roc-auc score calculation
roc_auc_score(df['march'],df.p)

0.6743147447289237

In [19]:
test = df.loc[pd.isnull(df.UserId) == True]
test

,user_ext,login,Unnamed: 0,UserId,p,totweek,RentCount
5,30366857,79998207073,NaN,NaN,NaN,NaN,NaN
7,30420205,79611064132,NaN,NaN,NaN,NaN,NaN
16,24974738,79818896449,NaN,NaN,NaN,NaN,NaN
27,30454499,79253342655,NaN,NaN,NaN,NaN,NaN
29,30393932,79629908322,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
232589,1374507,79771037718,NaN,NaN,NaN,NaN,NaN
232590,25968968,79859598775,NaN,NaN,NaN,NaN,NaN
232596,25131452,79173908735,NaN,NaN,NaN,NaN,NaN
232604,30408994,79647100333,NaN,NaN,NaN,NaN,NaN


In [15]:
# Version 2.0 With new drivers
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation_v2/aggregated_probs_till_march2020.csv')
users = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v7.csv', low_memory = False)
users = users[['login', 'user_ext']]
# All drivers who drove during validation period
new = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation_v2/march_validation_period_drivers.csv')
df = pd.merge(new,df,left_on = 'user_ext', right_on = 'UserId', how = 'left')
df.p = df.p.fillna(0)

# Changing "phone" format from int64 to string (object) for successful dataframes merge
dtp.phone = dtp.phone.astype(str)
dtp = pd.merge(dtp, users, left_on = 'phone', right_on = 'login', how = 'left')

# Only guilty and unknown
dtp = dtp.loc[(dtp.Responsibility != 'Фейк')&(dtp.Responsibility != 'Угон')&(dtp.Responsibility != 'Не виновен')]

# Data formatting
dtp['user_ext'] = dtp.user_ext.fillna(0)
df.UserId = df.UserId.astype(int)

# Splitting dtp data into periods for validation
# March
dtp_march = dtp.loc[(dtp.dt >= '2020-03-18') & (dtp.dt <= '2020-03-29')]
dtp_march = dtp_march[['dt','user_ext']]
dtp_march['march'] = 1

# Marking users with accidents
df = pd.merge(df,dtp_march, left_on = 'UserId', right_on = 'user_ext', how = 'left')
df['march'] = df['march'].fillna(0)

# Running roc-auc score calculation
roc_auc_score(df['march'],df.p)

KeyboardInterrupt: 